In [1]:
import os, sys
import numpy as np
import torch
import argparse
from collections import OrderedDict
#sys
#from saltseg.models import salt_models
sys.path.append("/pdata/pdata27/T_DATA_ANALYTICS/Src/FaultNet")  ## include path of FautNet
import time

import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

 We use the 20 images in data/TestSet as a pure hold-out test set for iou comparison
 Note:
 1. Wu's model (Geophysics, 2019) used this images as validation set. But we still show them here and will show higher iou when these images are used as a pure holdout test-set by our models. For all our models (vnet and unet_res*) these 20 images are never used in training or validation but set-aside for pure test-set.
 
 2. iou computation and prediction scripts for our models are included here
 


In [2]:
def computeiou(preds, actuals):
    preds = preds>0.5
    actuals = actuals>0.5
    intersection = actuals & preds
    union = actuals | preds
    
    return intersection.sum()/(union.sum()+1e-8)

Make predictions using vnet or unet_res1. Do the following:
Set path to test images and models correctly to run. 
Assumes your machine has GPUs. Note models provided here are trained after wrapping with DataParallel. 

In [3]:
model_path="/pdata/pdata27/T_DATA_ANALYTICS/FAULT/SEN_MODELS/unet3d_wu_Noiter_NoiternetChain2_CE3D/UNET3D_WU_ITERCHAIN_2020_01_04_249.pkl"
model_arch="unet_3d_res1" #"vnet"   ## model architecture tag : vnet or unet_res1
test_image_dir="/pdata/pdata27/T_DATA_ANALYTICS/Src/FaultNet/data/TestSet/images/"  # path to test images in TestSet
test_label_dir="/pdata/pdata27/T_DATA_ANALYTICS/Src/FaultNet/data/TestSet/labels/"  # path to test labels in TestSet
wu_pred_dir="/pdata/pdata27/T_DATA_ANALYTICS/Src/FaultNet/data/Predictions/WU_model/"   # path to preds from the wu model
output_dir="/pdata/pdata27/T_DATA_ANALYTICS/Src/FaultNet/data/VNetPreds/"  ## write model prediction here

In [12]:
def make_preds(modelin, image_folder, label_folder, wu_folder, out_folder,save_flag=False):
    """
    make predictions on images in the image folder and write out to out_folder
    compute iou for each image and compare to the iou of predictions mad by the wu-model
    """
    toc=time.time(); count_files=0
    for files in os.listdir(image_folder):
        filename, ext = os.path.splitext(files)
        image_file = os.path.join(image_folder, filename) +".npy"
        label_file = os.path.join(label_folder, filename) +".npy"
        wu_file = os.path.join(wu_folder, filename) +".npy"

        image_load = np.load(image_file)
        label_load = np.load(label_file)
        wu_pred = np.load(wu_file); wu_pred[wu_pred>=0.9]=1; wu_pred[wu_pred!=1]=0

        label_load = label_load.transpose(2,1,0) # Z, X, Y
        wu_pred = wu_pred.transpose(2,1,0)       # Z, X, Y

        image_load = (image_load - np.mean(image_load))/np.std(image_load)  # normalization used in training
        image_load = image_load.transpose(2,1,0)  # Z, X, Y
        image_load = np.expand_dims(np.expand_dims(image_load,0),0)  # 1,1,Z,X,Y (B,C,Z,X,Y)
        with torch.no_grad():
            image_tensor = torch.from_numpy(image_load).float().cuda()
            output = modelin(image_tensor)
        logits = output['logits']
        
        #log_preds1 = F.softmax(logits, dim=1)
        #log_preds1_numpy = log_preds1.data.cpu().numpy()
        #log_preds1_numpy = log_preds1_numpy[0,:,:,:,:]
        #pred_argmax = np.argmax(log_preds1_numpy, axis=0).astype(np.float32)
        
        probits = F.softmax(logits,dim=1).data.cpu().numpy()
        pred_argmax =  np.argmax(probits[0,:,:,:,:], axis=0).astype(np.float32)
        iou_preds = computeiou(pred_argmax,label_load)
        iou_wu = computeiou(wu_pred,label_load)
        print(" ---- for filename %s iou preds: %f wu-model: %f ----"%(filename,iou_preds,iou_wu))
        if save_flag==True:
            output_file = os.path.join(out_folder,filename)+".npy"
            np.save(output_file, pred_argmax.transpose(2,1,0)) # save in X,Y,Z format same as test labels/images
        count_files +=1
    tic=time.time()
    print(" Done prediction for %d files in %f s" %(count_files, tic-toc))

load model, run predictions on one gpu (specified as gpu:0 here)

In [5]:
gpu_use="0"
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_use

from models import load_models
model=load_models.getModel(model_arch=model_arch)


 model arch specified by user is  unet_3d_res1
loading UNet3d with 1 Residual block, output will have 2 channels


/pdata/pdata27/T_DATA_ANALYTICS/Src/FaultNet/models/networks_other.py:42: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight.data, a=0, mode='fan_in')


remove dataparallel from model dict as models will be used on single gpu for prediction

In [6]:
curr_dict=torch.load(model_path, map_location=lambda storage, loc: storage)['model_state']
new_state_dict = OrderedDict()
for k, v in curr_dict.items():
    name = k[7:]
    new_state_dict[name] = v
    

In [7]:
model.load_state_dict(new_state_dict)
model.eval()

unet_3D_Res1(
  (dropout): Dropout3d(p=0.6)
  (lrelu): LeakyReLU(negative_slope=0.01, inplace)
  (conv0_1): Conv3d(1, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv0_2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (conv0_3): Sequential(
    (0): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (1): LeakyReLU(negative_slope=0.01, inplace)
    (2): Conv3d(16, 16, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  )
  (inorm0): InstanceNorm3d(16, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (conv1): BasicRes(
    (conv1): Conv3d(16, 32, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1))
    (norm_relu_conv): Sequential(
      (0): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (1): LeakyReLU(negative_slope=0.01, inplace)
      (2): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
 

set folder path where preds will be written out. Each pred will be written out with the same filename as the test image

In [13]:
save_flag=False  # if set to true preds will be saved in output_dir
make_preds(model, test_image_dir, test_label_dir, wu_pred_dir, output_dir, save_flag)

 ---- for filename 0 iou preds: 0.764095 wu-model: 0.695840 ----
 ---- for filename 10 iou preds: 0.788676 wu-model: 0.706450 ----
 ---- for filename 11 iou preds: 0.703346 wu-model: 0.632799 ----
 ---- for filename 12 iou preds: 0.664703 wu-model: 0.609656 ----
 ---- for filename 13 iou preds: 0.555531 wu-model: 0.493070 ----
 ---- for filename 14 iou preds: 0.658020 wu-model: 0.598125 ----
 ---- for filename 15 iou preds: 0.622319 wu-model: 0.561113 ----
 ---- for filename 16 iou preds: 0.683888 wu-model: 0.652235 ----
 ---- for filename 17 iou preds: 0.737977 wu-model: 0.672268 ----
 ---- for filename 18 iou preds: 0.671623 wu-model: 0.576642 ----
 ---- for filename 19 iou preds: 0.625399 wu-model: 0.529391 ----
 ---- for filename 1 iou preds: 0.750731 wu-model: 0.701766 ----
 ---- for filename 2 iou preds: 0.821693 wu-model: 0.774017 ----
 ---- for filename 3 iou preds: 0.737561 wu-model: 0.686416 ----
 ---- for filename 4 iou preds: 0.661090 wu-model: 0.622698 ----
 ---- for filen